In [1]:
#Importando pacotes
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
from fuzzyJoao import partial_ratio
from argparse import ArgumentParser
import subprocess
import pandas as pd
from joblib import Parallel, delayed
from IPython.display import clear_output, display
import multiprocessing
import time
import re

In [2]:
dados_texto = pd.read_csv("/scratch/derickmath/text_processing/2006-2010/exames_texto.csv")
dicionario = pd.read_csv("/scratch/derickmath/text_processing/2006-2010/DicionarioECG.csv", sep = ";")

time parallel for 200 lines = 123.9019660949707

time sequential for 200 lines = 3.2906479835510254

In [3]:
start_time = time.time()
text = []
for i in range(len(dados_texto["conteudo"])):
    time100it = time.time()
    print(dados_texto['conteudo'][i], file=open("aux.rtf", "w"))
    sh = subprocess.call(["sh", "/scratch/derickmath/text_processing/2006-2010/runrtf.sh"])
    file = open("aux.txt", encoding="UTF-8")
    text.append(file.read())
    clear_output(wait=True)
    if(i%100 == 0):
        print("it = ", i, len(dados_texto['conteudo']), i/len(dados_texto['conteudo']))
        print("time = ", time.time() - start_time)
        print("100 it time = ", time.time() - time100it)
print(time.time() - start_time)

KeyboardInterrupt: 

In [ ]:
dados_texto["conteudo_txt"] = text
dados_texto.to_csv('/scratch/derickmath/text_processing/2006-2010/exames_texto_txt__.csv', index = False)

In [3]:
dados_texto = pd.read_csv("/scratch/derickmath/text_processing/2006-2010/exames_texto_txt.csv")

In [46]:
#Função que recebe o dicionário e uma String e retorna uma lista com a similaridade de cada doença à String
def checarSimilaridadeDoencas(texto, dicionario, id_evento):
    listaScore = []
    for row in dicionario.itertuples():
        maiorScore = 0
        maiorScoreL = []
        for diag in row:
            if ((type(diag) is not int and type(diag) is not float) and (type(texto) is not float)):
                score = partial_ratio(diag.lower(), texto.lower())
                if score[3] > maiorScore: 
                    maiorScore = score[3]
                    maiorScoreL = score
        if (maiorScore > 60):
            listaScore.append([id_evento, row[1], maiorScoreL[0], maiorScoreL[1], maiorScoreL[2], maiorScoreL[3], texto])
#         else:
#             listaScore.append([id_evento, row[1], maiorScoreL[0], maiorScoreL[1], maiorScoreL[2], maiorScoreL[3], texto])
    return listaScore

In [47]:
len(dados_texto['conteudo_txt'])

432198

In [ ]:
num_cores = multiprocessing.cpu_count()
start_time = time.time()
resultados_match_p = Parallel(n_jobs = num_cores)(delayed(checarSimilaridadeDoencas)
                              (dados_texto['conteudo_txt'][i], dicionario, dados_texto['id'][i])
                    for i in range(len(dados_texto['conteudo_txt'])))
print(time.time() - start_time)

In [ ]:
print("oi")

In [30]:
result = []
resultado_n_vazio = [resultados_match_p[x] for x in range(len(resultados_match_p)) if resultados_match_p[x] != []]

for x in resultado_n_vazio:
    for y in x:
        result.append(y)
result = pd.DataFrame(result, columns = ["id", "class", "inicio", "fim", "trecho", "score", "texto"])

In [31]:
resultado_vazio = [resultados_match_p[x] for x in range(len(resultados_match_p)) if resultados_match_p[x] == []]

In [39]:
len(resultado_vazio)

196253

In [1]:
corte_classes = [84, 85, 96, 84, 97, 94, 81, 100, 83, 95, 83, 83, 81, 92, 95, 81, 85, 92, 84, 83, 86, 89, 83, 92, 93, 92, 83, 83, 87, 
                100, 92, 80, 84, 87, 91, 84, 100, 81, 100, 100, 88, 100, 81, 90, 100, 83, 90, 81, 86, 83, 96, 82, 97, 85, 90, 81, 81, 97,
                95, 97, 84, 81, 84, 89, 86, 89, 83, 95]
ordem_classes = ['Sobrecarga ventricular esquerda (critérios de Romhilt-Estes)',
 'Sobrecarga atrial esquerda', 'Sobrecarga biventricular',
 'Corrente de lesão subendocárdica', 'Bloqueio atrioventricular 2:1',
 'Bloqueio atrioventricular total', 'Bloqueio atrioventricular de 1° grau',
 'Sobrecarga ventricular esquerda( critérios de voltagem)',
 'ECG dentro dos limites da normalidade para idade e sexo ',
 'Sobrecarga biatrial', 'Possível inversão de posicionamento de eletrodos',
 'Extrassístoles supraventriculares', 'Desvio do eixo do QRS para esquerda',
 'Taquicardia supraventricular ', 'Taquicardia sinusal',
 'área eletricamente inativa', 'Bradicardia sinusal', 'Pausa sinusal',
 'Sobrecarga atrial direita', 'Bloqueio de ramo direito',
 'Extrassístoles ventriculares' ,'Intervalo QT prolongado',
 'Arritmia sinusal', 'Dextrocardia ', 'Flutter atrial',
 'Bloqueio de ramo esquerdo',
 'Alterações primárias da repolarização ventricular ',
 'Isquemia subendocárdica', 'Ritmo juncional',
 'Sobrecarga ventricular direita', 'Bloqueio atrioventricular avançado',
 'Bloqueio divisional posteroinferior do ramo esquerdo',
 'Sobrecarga ventricular esquerda (critérios de voltagem)',
 'Bloqueio atrioventricular de 2° grau Mobitz I',
 'Bloqueio atrioventricular de 2° grau Mobitz II',
 'Pré-excitação ventricular tipo Wolff-Parkinson-White',
 'Sistema de estimulação cardíaca normofuncionante',
 'Alterações da repolarização ventricular atribuídas à ação digitálica',
 'Bloqueio de ramo direito e bloqueio divisional  posteroinferior do ramo esquerdo',
 'Progressão lenta de R nas derivações precordiais ',
 'Corrente de lesão subepicárdica - provável infarto agudo do miocárdio com supradesnivelamento de ST',
 'Suspeita de Síndrome de  Brugada, repetir V1-V2 em derivações superiores',
 'Desvio do eixo do QRS para direita', 'Ritmo atrial ectópico',
 'Intervalo PR curto', 'Ritmo atrial multifocal',
 'Sistema de estimulação cardíaca com disfunção',
 'Bloqueio divisional anterossuperior do ramo esquerdo',
 'Distúrbio de condução do ramo direito', 'Taquicardia atrial',
 'Batimento de escape atrial','Fibrilação atrial',
 'Taquicardia ventricular não sustentada',
 'Taquicardia ventricular sustentada',
 'Bloqueio de ramo direito e bloqueio divisional anterossuperior do ramo esquerdo',
 'Batimento de escape supraventricular', 'Batimento de escape ventricular ',
 'Batimento de escape juncional', 'Taquicardia juncional',
 'Taquicardia atrial multifocal ', 'Corrente de lesão subepicárdica',
 'Alterações inespecíficas da repolarização ventricular',
 'Alterações secundárias da repolarização ventricular',
 'Distúrbio de condução do ramo esquerdo', 'Repolarização precoce',
 'Ausência de sinal eletrocardiográfico que impede a análise',
 'Traçado com qualidade técnica insuficiente', 'Síndrome de Brugada']
print(len(ordem_classes), len(corte_classes))
##observações
##1 - bav1, regra para excluir todos que não forem "primeiro grau", cerca de 80 falsos positivos com a nota de corte 81
##2 - problema no "sve critérios de voltagem" (aparece 2 vezes, verificar) - cuidado com "(" no dicionário
##3 - problema no ritmo juncional
##4 - char estranho para mobitz I e II (33, 34)
##5 - padrão vazio para Sistema de estimulação cardíaca normofuncionante (36)
##6 - problema crase Alterações da repolarização ventricular atribuídas à ação digitálica (37)
##7 - padrão vazio para Bloqueio de ramo direito e bloqueio divisional  posteroinferior do ramo esquerdo (38)
##8 - padrão vazio para Progressão lenta de R nas derivações precordiais (39)
##9 - padrão vazio para Corrente de lesão subepicárdica - provável infarto agudo do miocárdio com supradesnivelamento de ST (40)
##10 - padrão vazio para (41)
##

68 68


In [ ]:
result[(result['class'] == ordem_classes[-1]) & (result['score'] <= 100) & (result['score'] >= 80)].groupby(["trecho", "score"]).id.nunique()

In [34]:
certos = [result[(result['class'] == ordem_classes[i]) & (result['score'] >= corte_classes[i])] for i in range(len(ordem_classes))]

In [38]:
for x in certos:
    #result[result['class'] == x].to_csv(x+".csv", index = False)
    if(len(x['class'].unique()) >= 1):
        x.to_excel("resultados/" + x['class'].unique()[0] + ".xlsx",index = False, header = True)
        x.to_csv("resultados/" + x['class'].unique()[0] + ".csv",index = False, header = True)

In [ ]:
output_file = "saida_lstm_model_10"

class_dim = 3
k = 10

#number of iteractions 
epochs = 10 
batch_size = 256
training_size = 1
#max number of words in embedding
embed_dim = 512
lstm_out_dim = 256
top_words = 7500
sentiment = []
text = []
lstm_input = []
####
file_out = open(output_file, 'w')
######

with open(dado_entrada_lstm_c) as file:
    lstm_input_data1 = [x.split("\t") for x in file.read().split("\n")]
size1 = len(lstm_input_data1)
with open(dado_entrada_lstm_e) as file:
    lstm_input_data2 = next_n_lines(file, size1)
    
size2 = len(lstm_input_data2)
lstm_input_data = lstm_input_data1 + lstm_input_data2
size = len(lstm_input_data)
print(size1, size2, size)
for i in range(size):
    lstm_input.append([ lstm_input_data[i][0] , re.sub(' +',' ',re.sub("[^a-zA-Z 0-9;á-Ź]+", " ", str(lstm_input_data[i]))).split(";")])
    sentiment.append(lstm_input[i][0])
    text.append(lstm_input[i][1])
tokenizer = Tokenizer(num_words = top_words, split = ' ')
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)
X_train = X
Y_train = sentiment
folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(X, sentiment))

In [ ]:
shape = pad_sequences(X).shape[1]
model = Sequential()
model.add(Embedding(top_words, embed_dim,input_length = shape))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(class_dim, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
X = pad_sequences(X)
Y = pd.get_dummies(sentiment).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = (1-training_size), random_state = 42)
model.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose = 2)

In [ ]:
with open(dado_saida_d) as file:
    lstm_teste = [x.split("\t") for x in file.read().split("\n")]
size = len(lstm_teste)
sentiment_test = []
text_test = []
lstm_input_test = []
for i in range(size - 1):
    lstm_input_test.append([ lstm_teste[i][0] , re.sub(' +',' ',re.sub("[^a-zA-Z 0-9;á-Ź]+", " ", str(lstm_teste[i]))).split(";")])
    sentiment_test.append(lstm_input_test[i][0])
    text_test.append(lstm_input_test[i][1])

#tokenizer.fit_on_texts(text_test)
X = tokenizer.texts_to_sequences(text_test)
X = pad_sequences(X, maxlen=12)
X_test = X
Y_test = sentiment_test